<a href="https://colab.research.google.com/github/aso1801028/AI_teach2020/blob/master/2020AI0305_fishvideo_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
##魚の画像を学習して動画解析に利用する
import urllib.request as req
url="https://github.com/masatokg/sample_photo/raw/master/fish.zip"
save_file = "fish.zip"
req.urlretrieve(url,save_file)

import urllib.request as req
url="https://github.com/masatokg/sample_photo/raw/master/nofish.zip"
save_file = "nofish.zip"
req.urlretrieve(url,save_file)


!unzip -o -q "./fish.zip"
!unzip -o -q "./nofish.zip"

In [11]:
###
##
import cv2
import os,glob,pickle
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score


In [12]:
#
image_size = (64,32)
path = "/content"
path_fish = path + "/fish"
path_nofish = "/nofish"
x=[]
y=[]
print(path_fish)
print(path_nofish)

/content/fish
/nofish


In [13]:
###
#画像データを読み込んで説明変数、目的変数に追加する処理の関数を定義
def read_dir(path,label,x,y,image_size):
  #正規表現
  files = glob.glob(path + "/*.jpg")
  print( files )
  for f in files:
    #画像データを読み込む
    img = cv2.imread(f)
    img = cv2.resize(img, image_size)
    img_data = img.reshape( -1, )
    x.append( img_data )
    y.append(label)


In [14]:
#
read_dir(path_nofish,0,x,y, image_size)
read_dir(path_fish,1,x,y, image_size)

#
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

#データを学習用とテスト検証用に分割する
clf=RandomForestClassifier()
clf.fit(x_train,y_train)

#ランダムフォレストアルゴリズムで学習モデルを生成、学習させる
y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred))

#精度の確認
y_pred = clf.predict(x_test)
print("学習制度:",accuracy_score(y_test,y_pred))

#学習モデルのメモリ状態を保存
with open("fish.pkl","wb") as fp:
  pickle.dump(clf,fp)

[]
['/content/fish/1436.jpg', '/content/fish/213.jpg', '/content/fish/1232.jpg', '/content/fish/1286.jpg', '/content/fish/248.jpg', '/content/fish/1443.jpg', '/content/fish/1326.jpg', '/content/fish/1280.jpg', '/content/fish/1394.jpg', '/content/fish/3816.jpg', '/content/fish/2836.jpg', '/content/fish/1980.jpg', '/content/fish/239.jpg', '/content/fish/4014.jpg', '/content/fish/2059.jpg', '/content/fish/2850.jpg', '/content/fish/1435.jpg', '/content/fish/3804.jpg', '/content/fish/1165.jpg', '/content/fish/1346.jpg', '/content/fish/1283.jpg', '/content/fish/1339.jpg', '/content/fish/2785.jpg', '/content/fish/3811.jpg', '/content/fish/3964.jpg', '/content/fish/1330.jpg', '/content/fish/2858.jpg', '/content/fish/1935.jpg', '/content/fish/2814.jpg', '/content/fish/1352.jpg', '/content/fish/3234.jpg', '/content/fish/1154.jpg', '/content/fish/2982.jpg', '/content/fish/4030.jpg', '/content/fish/1944.jpg', '/content/fish/1263.jpg', '/content/fish/223.jpg', '/content/fish/2874.jpg', '/content/fi

In [15]:
##学習モデルを利用して動画から魚の画像を検出
import cv2, os, copy, pickle
from google.colab.patches import cv2_imshow

#学習済みモデルをプログラムに読み込む
with open ("fish.pkl","rb") as fp:
  clf = pickle.load(fp)
#withおわり

#解析用画像出力でいれくとり用文字列
output_dir = "./bestshot"
#解析画像出力でいれくとりが存在しなければ作成
if not os.path.isdir(output_dir):os.mkdir(output_dir)
#以下でも同じことができる
#os.makedirs(output_dir,exist_ok=True)

#各処理用変数を定義
img_last = None#前回の画像データ用変数
fish_th = 3#画像を出力するかどうかの関数（1枚当たりの魚の検出数）
count = 0
frame_count = 0

In [16]:
###動画ファイルの読み込み処理
import urllib.request as req
url="https://github.com/masatokg/sample_photo/raw/master/fish.mp4"
save_file = "fish.zip"
req.urlretrieve(url,save_file)

('fish.zip', <http.client.HTTPMessage at 0x7fb27a13b198>)

In [22]:


#####################################

#ファイルをダウンロードして保存
cap = cv2.VideoCapture(save_file)
#
while(True):
  #
  is_ok,frame = cap.read()
  #
  if not is_ok:
    print("end.")
    break
  #
  #
  frame = cv2.resize(frame,(640,360))
  frame2 = copy.copy(frame)
  frame_count += 1
  #
  gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
  #
  gray = cv2.GaussianBlur(gray,(15,15),0)
  #
  img_b = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)[1]

  #
  if not img_last is None:
    #
    frame_diff = cv2.absdiff(img_last,img_b)

    #差分の特徴
    cnts = cv2.findContours(frame_diff,
                             cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)[0]
    #
    fish_count = 0
    for pt in cnts:
      #
      x,y,w,h  = cv2.boundingRect(pt)
      if w<100 or w>500:continue #


      imgex = frame[y:y+h, x:x+w]
      imgex = cv2.resize(imgex,(64,32))#
      image_data = imgex.reshape(-1,)
      #
      pred_y = clf.predict([image_data])
      #魚と判定出来たら領域に枠をつけてframe2という画像にする
      if pred_y[0] == 1:
        fish_count +=1
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)#Greenの枠
      #if文終わり
    #for文終わり
    #魚が映った画像を
    if fish_count > fish_th:
      fname = output_dir + "/fish" + str(count) + ".jpg"
      cv2.imwrite(fname,frame)
      count += 1
    #ifend

  #負荷が重いのでコメントアウト
  #
  #
  img_last = img_b#
  #
cap.release()#
print("ok",count,"/",frame_count,"/枚数検出")

end.
ok 409 / 1998 /枚数検出


In [23]:
#画像のダウンロード 内容を少し書き換えること
! zip -r /content/bestshot.zip /content/bestshot

from google.colab import files
files.download("/content/bestshot.zip")

ストリーミング出力は最後の 5000 行に切り捨てられました。
  adding: content/bestshot/226174.jpg (deflated 34%)
  adding: content/bestshot/187902.jpg (deflated 23%)
  adding: content/bestshot/156398.jpg (deflated 34%)
  adding: content/bestshot/221542.jpg (deflated 34%)
  adding: content/bestshot/133452.jpg (deflated 29%)
  adding: content/bestshot/194791.jpg (deflated 32%)
  adding: content/bestshot/110856.jpg (deflated 31%)
  adding: content/bestshot/186995.jpg (deflated 34%)
  adding: content/bestshot/156146.jpg (deflated 30%)
  adding: content/bestshot/123694.jpg (deflated 34%)
  adding: content/bestshot/105252.jpg (deflated 33%)
  adding: content/bestshot/25757.jpg (deflated 34%)
  adding: content/bestshot/221112.jpg (deflated 34%)
  adding: content/bestshot/88254.jpg (deflated 34%)
  adding: content/bestshot/217683.jpg (deflated 33%)
  adding: content/bestshot/68230.jpg (deflated 33%)
  adding: content/bestshot/199332.jpg (deflated 20%)
  adding: content/bestshot/226954.jpg (deflated 31%)
  adding: content

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>